In [1]:
import json
from spacy.lang.en import English
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

In [2]:
with open("acronym.json") as f:
    ACRONYM = json.loads(f.read())
    
with open("longform.json") as f:
    LONGFORM = json.loads(f.read())

In [3]:
nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
matcher.add("ACRONYM", None, *list(nlp.pipe(ACRONYM)))

In [4]:
def acronym_component(doc):
    # Create an entity Span with the label 'GPE' for all matches
    matches = matcher(doc)
    doc.ents = [Span(doc, start, end, label="ACRONYM") for match_id, start, end in matches]
    return doc

In [5]:
# Add the component to the pipeline
nlp.add_pipe(acronym_component)

In [6]:
# Getter that looks up the span text in the dictionary of country capitals
get_longform = lambda span: LONGFORM.get(span.text.lower())

In [7]:
# Register the Span extension attribute 'capital' with the getter get_capital
Span.set_extension("longform", getter=get_longform, force=True)

In [13]:
# Process the text and print the entity text, label and capital attributes
doc = nlp("aif")

text = doc.text
for ent in doc.ents:
    text = text.replace(ent.text, ent._.longform)
print(text)

Account In Force


In [9]:
ACRONYM.append('bs')
with open('acronym.json', 'w') as outfile:  
    json.dump(ACRONYM, outfile)

In [10]:
LONGFORM.update({"bs": "Balance Sheet"})
with open('longform.json', 'w') as outfile:  
    json.dump(LONGFORM, outfile)